In [189]:
from bs4 import BeautifulSoup
import requests
import json

In [190]:
MENU_URL = 'https://www.mcdonalds.com/ua/uk-ua/eat/fullmenu.html'
PRODUCT_BASE_URL = 'https://www.mcdonalds.com/dnaapp/itemDetails?country=UA&language=uk&showLiveData=true&item='

In [191]:
def get_product_details(product_url):
    try:
        response = requests.get(product_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        data_product_id = soup.find('div', id="page-root")['data-product-id']
        response = requests.get(PRODUCT_BASE_URL + data_product_id)
        response.raise_for_status()

        product_data = json.loads(response.text)['item']
        result = {'name' : product_data['item_name'],
                  'description' : product_data['description'],
                  'calories' : product_data['nutrient_facts']['nutrient'][2]['value'],
                  'fats' : product_data['nutrient_facts']['nutrient'][3]['value'],
                  'carbs' : product_data['nutrient_facts']['nutrient'][5]['value'],
                  'proteins' : product_data['nutrient_facts']['nutrient'][7]['value'],
                  'unsaturated_fats' : product_data['nutrient_facts']['nutrient'][4]['value'],
                  'sugar' : product_data['nutrient_facts']['nutrient'][6]['value'],
                  'salt' : product_data['nutrient_facts']['nutrient'][8]['value'],
                  'portion' : product_data['nutrient_facts']['nutrient'][0]['value']
                  }

        return result

    except requests.exceptions.RequestException as e:
        print(f"Error fetching {product_url}: {e}")
        return None
    except Exception as e:
        print(f"Error parsing {product_url}: {e}")
        return None

In [192]:
response = requests.get(MENU_URL)
soup = BeautifulSoup(response.content, 'html.parser')

In [193]:
products_links = set()
for link in soup.find_all('a', class_='cmp-category__item-link'):
        href = link.get('href')
        if href and href.startswith('/ua/uk-ua/product/'):
            full_url = f"https://www.mcdonalds.com{href}"
            products_links.add(full_url)

In [194]:
products_data = []
for link in products_links:
        print(link)
        data = get_product_details(link)
        if data:
            products_data.append(data)


https://www.mcdonalds.com/ua/uk-ua/product/200028.html
https://www.mcdonalds.com/ua/uk-ua/product/200125.html
https://www.mcdonalds.com/ua/uk-ua/product/200242.html
https://www.mcdonalds.com/ua/uk-ua/product/200163.html
https://www.mcdonalds.com/ua/uk-ua/product/200044.html
https://www.mcdonalds.com/ua/uk-ua/product/200266.html
https://www.mcdonalds.com/ua/uk-ua/product/200403.html
https://www.mcdonalds.com/ua/uk-ua/product/200160.html
https://www.mcdonalds.com/ua/uk-ua/product/200341.html
https://www.mcdonalds.com/ua/uk-ua/product/200243.html
https://www.mcdonalds.com/ua/uk-ua/product/200031.html
https://www.mcdonalds.com/ua/uk-ua/product/200372.html
https://www.mcdonalds.com/ua/uk-ua/product/7318.html
https://www.mcdonalds.com/ua/uk-ua/product/200342.html
https://www.mcdonalds.com/ua/uk-ua/product/200119.html
https://www.mcdonalds.com/ua/uk-ua/product/200361.html
https://www.mcdonalds.com/ua/uk-ua/product/200051.html
https://www.mcdonalds.com/ua/uk-ua/product/200396.html
https://www.

In [195]:
with open('products.json', 'w', encoding='utf-8') as f:
    json.dump(products_data, f, ensure_ascii=False, indent=4)